In [1]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import urllib.parse
import pymongo
import pprint
import pandas as pd

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
# Define database and collection
db = client.stocks
collection = db.large_cap

In [4]:
url = 'https://www.barchart.com/stocks/most-active/price-volume-leaders?screener=large_cap'
response = requests.get(url)
print(response.status_code)

403


In [5]:
# searchterm = urllib.parse.quote('left bold plusIconTd elp')
# print(searchterm)

In [6]:
# SCRAPING 
# Set the executable path and initialize the chrome browser in splinter
executable_path = {"executable_path":"chromedriver.exe"}
browser = Browser("chrome", **executable_path, headless=False)
browser.visit("https://www.barchart.com/stocks/most-active/price-volume-leaders?screener=large_cap")

In [7]:
# Convert the browser html to a soup object 
soup = browser.html
stock_soup = BeautifulSoup(soup, 'html.parser')

In [8]:
#print(stock_soup.prettify())

In [9]:
stock_symbol = stock_soup.find("td", class_= "symbol text-left").text
print(stock_symbol)

         TSLA     


In [10]:
stock_name = stock_soup.find("td", class_= "symbolName text-left").text
print(stock_name)

       Tesla Inc      


In [11]:
stock_price = stock_soup.find("td", class_= "lastPrice").text
print(stock_price)

       537.92      


In [12]:
stock_volume = stock_soup.find("td", class_= "volume").text
print(stock_volume)

       28,992,250      


In [13]:
stock_change = stock_soup.find("td", class_= "percentChange down").text
print(stock_change)

       -1.35%      


In [14]:
stock_symbol_all = stock_soup.find_all("td", class_= "symbol text-left")
stock_name_all = stock_soup.find_all("td", class_= "symbolName text-left")
stock_price_all = stock_soup.find_all("td", class_= "lastPrice")
stock_volume_all = stock_soup.find_all("td", class_= "volume")
stock_change_all = stock_soup.find_all("td", class_= "priceChange down")


# print(stock_symbol_all)

# print(stock_name_all)

# print(stock_price_all)

# print(stock_volume_all)

# print(stock_change_all)

In [15]:
symbol =[]
for stock in stock_symbol_all:
    #print(stock.text)
    symbol_list = symbol.append(stock.text)
    #print(symbol)

In [16]:
price =[]
for stock in stock_price_all:
    #print(stock.text)
    price_list = price.append(stock.text)
    #print(price)

In [17]:
volume = []   
for vol in stock_volume_all:
    #print(vol.text)
    volume_list = volume.append(vol.text)
    #print(volume)

In [18]:
name = []   
for names in stock_name_all:
    #print(names.text)
    names_list = name.append(names.text)
    #print(name)

In [19]:
change = []   
for names in stock_change_all:
    #print(names.text)
    change_list = change.append(names.text)
    #print(change)

In [20]:
# Convert to Pandas dataframe and export in csv
df = pd.DataFrame({'stock_name':name, 'stock_symbol':symbol,'stock_price':price, 'stock_volume':volume}) 
df

,stock_name,stock_symbol,stock_price,stock_volume
0,Tesla Inc,TSLA,537.92,"28,992,250"
1,Apple Inc,AAPL,312.68,"40,464,199"
2,Amazon.com Inc,AMZN,"1,869.44","3,440,878"
3,Beyond Meat Inc,BYND,117.05,"50,608,688"
4,Alibaba Group Holding,BABA,226.49,"17,265,561"
5,Microsoft Corp,MSFT,162.13,"23,475,682"
6,JP Morgan Chase & Company,JPM,138.80,"24,905,977"
7,Facebook Inc,FB,219.06,"13,288,939"
8,Wells Fargo & Company,WFC,49.30,"56,667,266"
9,Netflix Inc,NFLX,338.69,"7,199,386"


In [23]:
df.to_csv('stocks_large_cap.csv', index=False, encoding='utf-8')

In [24]:
# Importing the dataframe into mongodb

data = df.to_dict(orient='records')  # Here's our added param..

collection.insert_many(data)